In [ ]:
# run before everything else
import os
import pandas as pd
import re
from collections import defaultdict
from collections import Counter

In [ ]:
# main data xlsx maker
def create_data(input_path, output_filename):
    INPUT_FOLDER_NAME = input_path + "\\tagged"
    OUTPUT_FILE_NAME = input_path + "\\" + output_filename + ".csv"

    def remove_SPACE():
        # Check if the directory exists
        if os.path.isdir(INPUT_FOLDER_NAME):
            # List all files in the directory
            files = os.listdir(INPUT_FOLDER_NAME)

            # Loop through each file
            for file_name in files:
                if file_name.endswith('.txt'):
                    file_path = os.path.join(INPUT_FOLDER_NAME, file_name)

                    # Read the content of the file with the detected encoding
                    with open(file_path, 'r', encoding="utf-8") as file:
                        content = file.read()

                    # Replace _SPACE_ with an empty string
                    modified_content = content.replace('_SPACE_', '')

                    # Write the modified content back to the file
                    with open(file_path, 'w', encoding="utf-8") as file:
                        file.write(modified_content)

    def analyze(input_folder):

        # no mas, tan
        intensifier_list = ["absolutamente", "bastante", "casi", "completamente", "demasiado", "estrictamente",
                "especialmente", "extremadamente", "harto", "increíblemente", "mucho",
                "muy", "plenamente", "realmente", "sumamente", "súper", "totalmente", "verdaderamente",
                "bien", "super", "supel", "re"]

        apocope_dict = {'gran': 'grande',
                        'buen': 'bueno',
                        'cien': 'ciento',
                        'cualquier': 'cualquiero',
                        'cualesquier': 'cualesquiero',
                        'algún': 'alguno',
                        'mal': 'malo',
                        'ningún': 'ninguno',
                        'primer': 'primero',
                        'tercer': 'tercero',
                        'un': 'uno'}

        forms_of_estar = ["estar", "estal",
                          "estoy", "estás", "está", "estamos", "estáis", "están",
                          "estaba", "estabas", "estábamos", "estabais", "estaban",
                          "estuve", "estuviste", "estuvo", "estuvimos", "estuvisteis", "estuvieron",
                          "estaré", "estarás", "estará", "estaremos", "estaréis", "estarán",
                          "estaría", "estarías", "estaría", "estaríamos", "estaríais", "estarían",
                          "esté", "estés", "estemos", "estéis", "estén",
                          "estuviera", "estuvieras", "estuviéramos", "estuvierais", "estuvieran",
                          "estuviese", "estuvieses", "estuviésemos", "estuvieseis", "estuviesen",
                          "estuviere", "estuvieres", "estuviéremos", "estuviereis", "estuvieren",
                          "estad"]

        forms_of_ser = ["ser", "sel",
                        "soy", "eres", "es", "somos", "sois", "son",
                        "era", "eras", "éramos", "erais", "eran",
                        "fui", "fuiste", "fue", "fuimos", "fuisteis", "fueron",
                        "seré", "serás", "será", "seremos", "seréis", "serán",
                        "sería", "serías", "sería", "seríamos", "seríais", "serían",
                        "sea", "seas", "seás", "seamos", "seáis", "sean",
                        "fuera", "fueras", "fuéramos", "fuerais", "fueran",
                        "fuese", "fueses", "fuésemos", "fueseis", "fuesen",
                        "fuere", "fueres", "fuéremos", "fuereis", "fueren",
                        "sé", "sed"]

        forms_of_parecer = ["parecer", "parecel",
                            "parezco", "pareces", "parecés", "parece", "parecemos", "parecéis", "parecen",
                            "parecía", "parecías", "parecíamos", "parecíais", "parecían",
                            "parecí", "pareciste", "pareció", "parecimos", "parecisteis", "parecieron",
                            "pareceré", "parecerás", "parecerá", "pareceremos", "pareceréis", "parecerán",
                            "parecería", "parecerías", "parecería", "pareceríamos", "pareceríais", "parecerían",
                            "parezca", "parezcas", "parezcás", "parezcamos", "parezcáis", "parezcan",
                            "pareciera", "parecieras", "pareciéramos", "parecierais", "parecieran",
                            "pareciese", "parecieses", "pareciésemos", "parecieseis", "pareciesen",
                            "pareciere", "parecieres", "pareciéremos", "pareciereis", "parecieren",
                            "paracé", "pareced"]

        copulas = forms_of_estar + forms_of_ser + forms_of_parecer

        # Get the current working directory in Colab
        colab_dir = "/content"

        # Construct the absolute path to the input folder
        input_folder_path = os.path.join(colab_dir, input_folder)

        # Create a list to store data frames
        data_frames = []

        # Iterate over marked files in the input folder
        for filename in os.listdir(input_folder_path):
            # SPEAKER
            pattern = re.compile(r'\.txt')
            speaker = pattern.sub('', "CART" + filename[:2])
            print("Started analyzing:", speaker)

            # Construct the absolute path to the marked file
            marked_file_path = os.path.join(input_folder_path, filename)

            # Initialize a list to store row data
            rows = []

            # Read the marked text from the file
            with open(marked_file_path, "r", encoding="utf-8") as file:
                # Process marked text line by line
                for line in file:
                    # Split the line into segments based on custom sentence-ending punctuation
                    segments = re.split(r'(?<=[,.!?]_PUNCT_[,.!?])', line)

                    # Initialize variables inside the inner loop
                    intensifier = None
                    adjective = None
                    lex_adjective = None
                    attributive = None
                    noun = None
                    double_intensified = None

                    # Process each segment
                    for segment in segments:
                        # Split the segment into words
                        words = segment.split()

                        # Process each word in the segment
                        for i, word in enumerate(words):
                            if "_ADJ_" in word and any(word.endswith(c) for c in 'oaeslndr') and not(word.endswith('mente')) and re.match(r'^[a-zA-Z_áéíóúüñÁÉÍÓÚÜÑ]+$', word):
                                # ADJECTIVO
                                adjective = re.search(r'^(.*)_ADJ_', word).group(1)
                                lex_adjective = re.search(r'_ADJ_(.+)', word).group(1)
                                # apocope corrections
                                if lex_adjective in apocope_dict:
                                    lex_adjective = apocope_dict[lex_adjective]
                                if lex_adjective.endswith("s"):
                                    lex_adjective = lex_adjective[:-1]
                                if lex_adjective.endswith("a"):
                                    lex_adjective = lex_adjective[:-1] + "o"
                                ####print(adjective)
                                ####print(lex_adjective)
                                # ATRIBUTIVO, NOMINAL
                                # noun adj
                                if i > 0 and "_NOUN_" in words[i-1] and not any(inter in words[i-1] for inter in intensifier_list):
                                    attributive = "attributive"
                                    noun = re.search(r'^(.*?)_', words[i-1]).group(1)
                                # adj noun
                                elif i < len(words) - 1 and "_NOUN_" in words[i+1]:
                                    attributive = "attributive"
                                    noun = re.search(r'^(.*?)_', words[i+1]).group(1)
                                # noun int adj
                                elif i > 1 and "_NOUN_" in words[i-2] and any(inter in words[i-1] for inter in intensifier_list):
                                    attributive = "attributive"
                                    noun = re.search(r'^(.*?)_', words[i-2]).group(1)
                                # cop int adj
                                elif i > 1 and any(form in words[i-2] for form in copulas):
                                    attributive = "predicative"
                                    noun = "/"
                                # cop adj
                                elif i > 0 and any(form in words[i-1] for form in copulas):
                                    attributive = "predicative"
                                    noun = "/"
                                else:
                                    attributive = "ambiguous"
                                    noun = "/"
                                if not re.match(r'^[a-zA-Z_áéíóúüñÁÉÍÓÚÜÑ/]+$', noun):
                                    attributive = "ambiguous"
                                    noun = "/"
                                ####print(noun)
                                # INTENSIFICADOR
                                if i > 0:
                                    prev_word_match = re.search(r'^(.*?)_', words[i-1])
                                    if prev_word_match:
                                        prev_word = prev_word_match.group(1)
                                        if prev_word in intensifier_list:
                                            intensifier = prev_word
                                        else:
                                            intensifier = '/'
                                    else:
                                        intensifier = '/'
                                else:
                                    intensifier = '/'
                                if adjective.endswith("ísimo") or adjective.endswith("ísima"):
                                    intensifier = '-ísimo'
                                if adjective.startswith("archi"):
                                    intensifier = 'archi-'
                                if intensifier == 'super' or intensifier == 'supel':
                                    intensifier = 'súper'
                                if adjective.startswith("re"):
                                    print(adjective)
                                ####print(intensifier)
                                # DOBLE_INT
                                if intensifier != '/' and i > 1:
                                    prev_prev_word_match = re.search(r'^(.*?)_', words[i-2])
                                    if prev_prev_word_match:
                                        prev_prev_word = prev_prev_word_match.group(1)
                                        if prev_prev_word in intensifier_list:
                                            double_intensified = prev_prev_word
                                        else:
                                            double_intensified = '/'
                                    else:
                                        double_intensified = '/'
                                else:
                                    double_intensified = '/'
                                # FRASE_COMPLETA
                                phrase_tagged = ' '.join(words)
                                # FRASE REAL
                                phrase = ' '.join(re.sub(r'_.*', '', word) for word in words)

                                row_data = {
                                    'ID': speaker,
                                    'Phr_tagged': phrase_tagged,
                                    'Phr': phrase,
                                    'Int': intensifier,
                                    'Double': double_intensified,
                                    'Adj': adjective,
                                    'Lex_Adj': lex_adjective,
                                    'Adj_type': attributive,
                                    'Noun': noun,
                                }
                                rows.append(row_data)

                    # Check if rows list is not empty
            if rows:
                # Create a DataFrame for the current file and append it to the list
                df = pd.DataFrame(rows)
                data_frames.append(df)
                print(f"Appended DataFrame for file: {filename}")
            else:
                print(f"No valid rows found in file: {filename}")

        # Concatenate all data frames in the list
        if data_frames:
            data = pd.concat(data_frames, ignore_index=True)
            # Save the DataFrame to a CSV file
            data.to_csv(input_path + "\\temp.csv", index=False)
            print("Successfully concatenated data frames")
        else:
            print("No data frames to concatenate")

            # Create a DataFrame for the current file and append it to the list
            data_frames.append(pd.DataFrame(rows))

        # Concatenate all data frames in the list
        data = pd.concat(data_frames, ignore_index=True)

        # Save the DataFrame to a CSV file
        data.to_csv(OUTPUT_FILE_NAME, index=False)

    def closest_verb_non_infinitive(row):
        phrase = row['Phr_tagged']
        intensifier_pos = phrase.find(row['Adj'])

        # Initialize minimum distances and closest matches
        min_distance_verb = float('inf')
        min_distance_aux = float('inf')
        closest_verb = None  # To store the closest verb
        closest_aux = None   # To store the closest auxiliary

        # Regular expression to find verb and auxiliary patterns
        pattern = r'\b(\w+)_(VERB|AUX)_(\w+)\b'  # Capture the first word

        # Use re.finditer to find all matches of the pattern
        for match in re.finditer(pattern, phrase):
            # Find the start and end of the match
            start_pos = match.start()
            end_pos = match.end()

            # Calculate distance as the distance to the center of the match
            match_center = (start_pos + end_pos) // 2
            distance = abs(match_center - intensifier_pos)

            # Check if the match is VERB or AUX and update accordingly
            if 'VERB' in match.group():
                if distance < min_distance_verb:
                    min_distance_verb = distance
                    closest_verb = match.group(1)  # Store the first word
            elif 'AUX' in match.group():
                if distance < min_distance_aux:
                    min_distance_aux = distance
                    closest_aux = match.group(1)  # Store the first word

        # Determine which is closer to the intensifier
        if closest_verb and closest_aux:
            if min_distance_verb < min_distance_aux:
                return closest_verb
            else:
                return closest_aux
        elif closest_verb:
            return closest_verb
        elif closest_aux:
            return closest_aux
        else:
            return '/'  # Return '/' if no relevant forms are found

    def closest_verb_infinitive(row):
        phrase = row['Phr_tagged']
        intensifier_pos = phrase.find(row['Adj'])

        # Initialize minimum distances and closest matches
        min_distance_verb = float('inf')
        min_distance_aux = float('inf')
        closest_verb = None  # To store the closest verb
        closest_aux = None   # To store the closest auxiliary

        # Regular expression to find verb and auxiliary patterns
        pattern = r'\b(\w+)_(VERB|AUX)_(\w+)\b'  # Capture both words

        # Use re.finditer to find all matches of the pattern
        for match in re.finditer(pattern, phrase):
            # Find the start and end of the match
            start_pos = match.start()
            end_pos = match.end()

            # Calculate distance as the distance to the center of the match
            match_center = (start_pos + end_pos) // 2
            distance = abs(match_center - intensifier_pos)

            # Check if the match is VERB or AUX and update accordingly
            if 'VERB' in match.group():
                if distance < min_distance_verb:
                    min_distance_verb = distance
                    closest_verb = match.group(3)  # Store the second word (infinitive)
            elif 'AUX' in match.group():
                if distance < min_distance_aux:
                    min_distance_aux = distance
                    closest_aux = match.group(3)  # Store the second word (infinitive)

        # Determine which is closer to the intensifier
        if closest_verb and closest_aux:
            if min_distance_verb < min_distance_aux:
                return closest_verb
            else:
                return closest_aux
        elif closest_verb:
            return closest_verb
        elif closest_aux:
            return closest_aux
        else:
            return '/'  # Return '/' if no relevant forms are found

    remove_SPACE()

    # Call the function with the marked input folder path
    analyze(INPUT_FOLDER_NAME)

    # Read the original CSV file into a DataFrame
    df = pd.read_csv(OUTPUT_FILE_NAME)

    # Apply the function to create the new column
    df['Verb'] = df.apply(closest_verb_non_infinitive, axis=1)
    df['Inf_Verb'] = df.apply(closest_verb_infinitive, axis=1)

    # Save the modified DataFrame back to a CSV and XLSX
    df.to_csv(OUTPUT_FILE_NAME, index=False)
    df = pd.read_csv(OUTPUT_FILE_NAME)
    output_file_name = OUTPUT_FILE_NAME[:-4] + '.xlsx'
    df.to_excel(output_file_name, index=False)

    # Check if the file exists before trying to delete it
    os.remove(input_path + "\\" + output_filename + ".csv")
    os.remove(input_path + "\\temp.csv")

    print("Successfully created data")
create_data("C:\\ClassStuff\\4005\\datos", "intensifier_data_CART_101815")


In [3]:
cart_data = "..\\datos\\intensifier_data_CART_101815.xlsx"

cart_df = pd.read_excel(cart_data)

In [ ]:
# make numbered based on speaker col
def add_number(df, output_file):
    def numero(row, count_dict={}):
        # Split the original label to get the speaker number
        speaker_number = row['ID']

        # Update the count for this speaker number
        count_dict[speaker_number] = count_dict.get(speaker_number, 0) + 1

        # Generate the new label
        new_label = f'{speaker_number}_{count_dict[speaker_number]:02d}'
        return new_label
    df['Number'] = df.apply(numero, axis=1)
    df.to_excel(output_file)

add_number(cart_df, cart_data)

In [ ]:
# fix -o to o|a|s etc.
def lex_adj_fixer(df, output_file):
    # Assuming the column name is 'word', adjust if necessary
    words = df['Adj'].tolist()

    # Dictionaries to store grouped words
    word_groups = defaultdict(set)
    a_groups = defaultdict(set)
    e_groups = defaultdict(set)
    es_groups = defaultdict(set)
    word_counts = defaultdict(int)

    # Identify and group words by root while counting occurrences
    for word in words:
        word_counts[word] += 1  # Track occurrences

        match_o_a = re.match(r"(.+?)(o|a|os|as)$", word)
        match_a = re.match(r"(.+?)(es|a|as)$", word)
        match_e = re.match(r"(.+?)(e|es)$", word)
        match_es = re.match(r"(.+?)(es)$", word)

        if match_o_a:
            root, ending = match_o_a.groups()
            word_groups[root].add(ending)
        elif match_a:
            root, ending = match_a.groups()
            word_groups[root].add(ending)
        elif match_e:
            root, ending = match_e.groups()
            e_groups[root].add(ending)
        elif match_es:
            root, ending = match_es.groups()
            es_groups[root].add(ending)
        else:
            es_groups[word].add("")  # Mark singular words

    # Dictionary to store final merged word mappings
    merged_map = {}

    # Process "o, a, os, as" groups
    for root, endings in word_groups.items():
        sorted_endings = sorted(endings)
        merged = root + "|".join(sorted_endings) if sorted_endings else root
        merged_map.update({root + ending: merged for ending in endings})  # Map all variations to merged form

    # Process "es, a, as" groups
    for root, endings in a_groups.items():
        sorted_endings = sorted(endings)
        merged = root + "|".join(sorted_endings) if sorted_endings else root
        merged_map.update({root + ending: merged for ending in endings})  # Map all variations to merged form

    # Process "e, es" groups
    for root, endings in e_groups.items():
        merged = f"{root}e|s" if endings == {"e", "es"} else f"{root}{'|'.join(sorted(endings))}"
        merged_map.update({root + ending: merged for ending in endings})

    # Process singular ("") and plural ("es") groups
    for root, endings in es_groups.items():
        if endings == {"", "es"}:
            merged = f"{root}|es"
        elif endings == {"es"}:
            merged = f"{root}es"
        else:
            merged = root
        merged_map[root] = merged
        merged_map[root + "es"] = merged  # Handle plural form

    # Generate final expanded list
    expanded_merged_words = []
    for word in words:
        expanded_merged_words.append(merged_map.get(word, word))  # Replace with merged form

    # Save to Excel
    output_df = pd.DataFrame({"merged_word": expanded_merged_words})
    output_df.to_excel(output_file, index=False)  # Save as Excel file

    print(f"Merged words saved to {output_file}")

lex_adj_fixer(cart_df, cart_data) # NOT USED RN

Merged words saved to ..\datos\intensifier_data_CART_101815.xlsx


In [6]:
# lex adjs by freq
def lex_adjs_freq(df):
    # Assuming the column of words is named 'words'
    words = df['Lex_Adj'].dropna().tolist()  # Remove NaNs and convert to list

    # Count word frequencies
    word_counts = Counter(words)

    # Total number of words
    total_words = sum(word_counts.values())

    # Create a sorted list of words with their percentages
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print results
    place = 0
    for word, count in sorted_word_counts:
        percentage = (count / total_words) * 100
        if percentage >= 1.0: ## ! CHANGE HERE TO CHANGE RANGE SHOWN'
            place += 1
            print(f"{place}th {word}: {count} ({percentage:.2f}%)")

lex_adjs_freq(cart_df)

1th bueno: 88 (11.91%)
2th primero: 31 (4.19%)
3th único: 20 (2.71%)
4th mejor: 16 (2.17%)
5th mismo: 16 (2.17%)
6th grande: 15 (2.03%)
7th bonito: 13 (1.76%)
8th pequeño: 12 (1.62%)
9th normal: 12 (1.62%)
10th último: 12 (1.62%)
11th económico: 11 (1.49%)
12th malo: 10 (1.35%)
13th solo: 9 (1.22%)
14th cristiano: 9 (1.22%)
15th segundo: 8 (1.08%)


In [7]:
# add syl col
def count_syllables(word):
    # Remove silent 'u' in gue/gui/que/qui unless it has a diaeresis (ü)
    word = re.sub(r'gue|gui|que|qui', lambda m: m.group(0).replace('u', ''), word)

    # Count vowel groups (as a simple approximation of syllables)
    syllables = re.findall(r'[aeiouáéíóúü]+', word)
    
    return len(syllables)

def addSyllableCol(df):
    # Create a new column for the number of syllables in the text
    # Does not account for diphthongs or triphthongs
    df['Syllable'] = df['Adj'].apply(count_syllables)
    return df

df = addSyllableCol(cart_df)
df.to_excel(cart_data)